In [67]:
import pandas as pd

# Memuat data alamat
alamat_df = pd.read_excel('Data Alamat.xlsx', sheet_name='SMT')
# Backup alamat untuk mencegah kehilangan nilai saat proses
if 'ALAMAT_backup' not in alamat_df.columns:
    alamat_df['ALAMAT_backup'] = alamat_df['ALAMAT']
print(alamat_df.head())
print(f"Baris: {len(alamat_df)} | Kolom: {list(alamat_df.columns)}")


                                              ALAMAT           kecamatan  \
0  SRINANTI RT/RW 005/003 DESA SUNGAI GERONG KEC ...    KEC. BANYUASIN.I   
1  JL. KELAPA LK IV KEL. SUKA MAJU KEC. BINJAI BA...  KEC. BINJAI BARAT    
2  UJUNG PADANG RT/RW -/- KEL. SASAK KEC. SASAK R...                 NaN   
3  DESA ONOWAEMBO KEC. GUNUNGSITOLI IDANOI KOTA G...                 NaN   
4  JL. HUTA DOLOK III RT/RW 000/000 KEL. LBN DOLO...                 NaN   

           kabupaten        area_polreg Province_polreg  \
0          BANYUASIN          BANYUASIN  South Sumatera   
1             BINJAI             Binjai  North Sumatera   
2  KAB PASAMAN BARAT  KAB PASAMAN BARAT   West Sumatera   
3      GUNUNG SITOLI               Nias  North Sumatera   
4               TOBA            Samosir  North Sumatera   

                                       ALAMAT_backup  
0  SRINANTI RT/RW 005/003 DESA SUNGAI GERONG KEC ...  
1  JL. KELAPA LK IV KEL. SUKA MAJU KEC. BINJAI BA...  
2  UJUNG PADANG RT/RW -

In [68]:
# Memuat data referensi kecamatan (BPS 2017)
kecamatan_df = pd.read_excel('Daftar Kecamatan Indonesia BPS 2017.xlsx')
# Filter data referensi kecamatan hanya pulau sumatera saja (ACEH, Sumatera Utara, Sumatera Barat, Riau, Kep. Riau, Kep. Sumatera Utara, Kep. Sumatera Barat)
kecamatan_df = kecamatan_df[kecamatan_df['Provinsi'].isin(['ACEH', 'SUMATERA UTARA', 'SUMATERA BARAT', 'RIAU', 'KEPULAUAN RIAU','JAMBI', 'BENGKULU', 'SUMATERA SELATAN', 'LAMPUNG', 'KEPULAUAN BANGKA BELITUNG'])]
print(kecamatan_df.head())
print(f"Baris: {len(kecamatan_df)} | Kolom: {list(kecamatan_df.columns)}")

              Kecamatan  Kecamatan_ID        Kabupaten  Kabupaten_ID  \
0  2 X 11 ENAM LINGKUNG            50  PADANG PARIAMAN             6   
3         ABUNG  KUNANG            32    LAMPUNG UTARA             6   
4         ABUNG SELATAN            50    LAMPUNG UTARA             6   
5          ABUNG SEMULI            51    LAMPUNG UTARA             6   
7          ABUNG TENGAH            31    LAMPUNG UTARA             6   

         Provinsi  Provinsi_ID  
0  SUMATERA BARAT           13  
3         LAMPUNG           18  
4         LAMPUNG           18  
5         LAMPUNG           18  
7         LAMPUNG           18  
Baris: 1861 | Kolom: ['Kecamatan', 'Kecamatan_ID', 'Kabupaten', 'Kabupaten_ID', 'Provinsi', 'Provinsi_ID']


In [69]:
import re
from rapidfuzz import process, fuzz

STOP_TOKENS = {
    'KABUPATEN', 'KAB', 'KOTA', 'KECAMATAN', 'KEC', 'PROVINSI', 'PROV',
    'KELURAHAN', 'KEL', 'DESA', 'DS', 'RT', 'RW', 'JL', 'JLN', 'JALAN',
    'LINGKUNGAN', 'LK', 'NO', 'NOMOR', 'GANG', 'GG', 'LAINNYA', 'PS', 'PASAR',
    'BLOK', 'DUSUN', 'DSN', 'KOMP', 'COMPLEX'
}

ABBREV_MAP = {
    'KAB.': 'KABUPATEN',
    'KAB ': 'KABUPATEN ',
    'KAB-': 'KABUPATEN ',
    'KOTA ': 'KOTA ',
    'KOT.': 'KOTA',
    'KOTAAdm': 'KOTA',
    'KEC.': 'KECAMATAN',
    'KEC ': 'KECAMATAN ',
    'KCAMATAN': 'KECAMATAN',
    'KCAM.': 'KECAMATAN',
    'KCM.': 'KECAMATAN',
    'KEM.': 'KECAMATAN',
    'D.K.': 'DKI',
}

PUNCT_PATTERN = re.compile(r"[.,/\\-]")
MULTISPACE_PATTERN = re.compile(r"\s+")


def normalize_text(text: str) -> str:
    text = text.upper()
    for src, dst in ABBREV_MAP.items():
        text = text.replace(src, dst)
    text = PUNCT_PATTERN.sub(' ', text)
    text = MULTISPACE_PATTERN.sub(' ', text)
    return text.strip()


def clean_candidate(candidate: str) -> str:
    tokens = []
    for token in candidate.strip().split():
        if token in STOP_TOKENS:
            break
        tokens.append(token)
    return ' '.join(tokens).strip()


def extract_keyword_segment(text: str, keyword: str) -> str:
    keyword = keyword.upper()
    if keyword not in text:
        return ''
    start = text.index(keyword) + len(keyword)
    tail = text[start:]
    # Berhenti ketika menemukan token kata kunci lain
    split = re.split(r"\b(KABUPATEN|KOTA|KECAMATAN|PROVINSI|KELURAHAN|DESA|RT|RW|JL|JALAN|LINGKUNGAN|LK)\b", tail, maxsplit=1)
    candidate = split[0] if split else tail
    return clean_candidate(candidate)


def fuzzy_match(name: str, choices, scorer=fuzz.WRatio, score_cutoff=80, processor=None):
    if not name:
        return ''
    match = process.extractOne(
        name,
        choices,
        scorer=scorer,
        score_cutoff=score_cutoff,
        processor=processor,
    )
    return match[0] if match else ''

In [70]:
# Prepare normalized reference data
kecamatan_df['Kecamatan_norm'] = kecamatan_df['Kecamatan'].apply(normalize_text)

kabupaten_df = kecamatan_df[['Kabupaten']].drop_duplicates().copy()
kabupaten_df['Kabupaten_norm'] = kabupaten_df['Kabupaten'].apply(normalize_text)

kecamatan_choices = list(dict.fromkeys(kecamatan_df['Kecamatan']))
kabupaten_choices = list(dict.fromkeys(kabupaten_df['Kabupaten']))

kecamatan_norm_map = {}
for original, norm in zip(kecamatan_df['Kecamatan'], kecamatan_df['Kecamatan_norm']):
    kecamatan_norm_map.setdefault(norm, original)

kabupaten_norm_map = {}
for original, norm in zip(kabupaten_df['Kabupaten'], kabupaten_df['Kabupaten_norm']):
    kabupaten_norm_map.setdefault(norm, original)

kecamatan_by_kabupaten = {}
for kecamatan, kec_norm, kabupaten in zip(
    kecamatan_df['Kecamatan'],
    kecamatan_df['Kecamatan_norm'],
    kecamatan_df['Kabupaten'],
):
    info = kecamatan_by_kabupaten.setdefault(kabupaten, {'choices': [], 'norm_map': {}})
    if kecamatan not in info['choices']:
        info['choices'].append(kecamatan)
    info['norm_map'].setdefault(kec_norm, kecamatan)


def contains_reference(text: str, norm_map: dict) -> str:
    for norm_name, original in norm_map.items():
        if norm_name and norm_name in text:
            return original
    return ''


def standardize_kabupaten(name) -> str:
    if name is None:
        return ''
    name_str = str(name).strip()
    if not name_str or name_str.lower() == 'nan':
        return ''
    name_norm = normalize_text(name_str)
    if name_norm in kabupaten_norm_map:
        return kabupaten_norm_map[name_norm]
    match = fuzzy_match(name_str, kabupaten_choices, processor=normalize_text, score_cutoff=88)
    return match or ''


def resolve_kecamatan(text: str, kabupaten_ref: str = '') -> str:
    info = None
    if kabupaten_ref:
        info = kecamatan_by_kabupaten.get(kabupaten_ref)
        if not info:
            kab_norm = normalize_text(kabupaten_ref)
            official = kabupaten_norm_map.get(kab_norm)
            if official:
                info = kecamatan_by_kabupaten.get(official)

    norm_map = info['norm_map'] if info else kecamatan_norm_map
    choices = info['choices'] if info else kecamatan_choices

    # Prioritas: cari di segment setelah kata KECAMATAN
    candidate = extract_keyword_segment(text, 'KECAMATAN')
    if candidate:
        candidate_norm = normalize_text(candidate)
        if candidate_norm in norm_map:
            return norm_map[candidate_norm]
        match = fuzzy_match(candidate, choices, processor=normalize_text)
        if match:
            return match
    
    # Jika tidak ada kata KECAMATAN, cari di bagian sebelum KABUPATEN/KOTA
    # untuk menghindari mencari di bagian kabupaten
    text_before_kab = text
    for keyword in ('KABUPATEN', 'KOTA'):
        if keyword in text:
            idx = text.index(keyword)
            text_before_kab = text[:idx].strip()
            break
    
    # Cari di bagian sebelum kabupaten saja
    direct = contains_reference(text_before_kab, norm_map)
    if direct:
        return direct
    match = fuzzy_match(text_before_kab, choices, processor=normalize_text, score_cutoff=85)
    return match or ''


def standardize_kecamatan(name, kabupaten_ref: str = '') -> str:
    if is_nullish(name):
        return ''
    name_str = str(name).strip()
    info = None
    if kabupaten_ref:
        info = kecamatan_by_kabupaten.get(kabupaten_ref)
        if not info:
            kab_norm = normalize_text(kabupaten_ref)
            official = kabupaten_norm_map.get(kab_norm)
            if official:
                info = kecamatan_by_kabupaten.get(official)
    norm_map = info['norm_map'] if info else kecamatan_norm_map
    choices = info['choices'] if info else kecamatan_choices
    name_norm = normalize_text(name_str)
    if name_norm in norm_map:
        return norm_map[name_norm]
    match = fuzzy_match(name_str, choices, processor=normalize_text, score_cutoff=85)
    return match or name_str


In [71]:
import numpy as np


def is_nullish(value) -> bool:
    if value is None or (isinstance(value, float) and np.isnan(value)):
        return True
    if isinstance(value, str):
        stripped = value.strip()
        return stripped == ''
    return False


alamat_df['alamat_norm'] = alamat_df['ALAMAT'].astype(str).apply(normalize_text)

kabupaten_standard = alamat_df['kabupaten'].apply(standardize_kabupaten)
kabupaten_matched_mask = kabupaten_standard.apply(lambda x: not is_nullish(x))
alamat_df.loc[kabupaten_matched_mask, 'kabupaten'] = kabupaten_standard[kabupaten_matched_mask]

kabupaten_standardized = kabupaten_matched_mask.sum()
kabupaten_unmatched = len(alamat_df) - kabupaten_standardized

needs_kec = alamat_df['kecamatan'].apply(is_nullish)

updated_kec = alamat_df['kecamatan'].astype(object).copy()

for row in alamat_df.loc[needs_kec, ['alamat_norm', 'kabupaten']].itertuples():
    idx = row.Index
    text = row.alamat_norm
    kabupaten_ref = row.kabupaten if not is_nullish(row.kabupaten) else ''
    resolved_kec = resolve_kecamatan(text, kabupaten_ref)
    if resolved_kec:
        updated_kec.at[idx] = resolved_kec

alamat_df['kecamatan'] = updated_kec

# Normalisasi kecamatan menggunakan anchor kabupaten (BPS)
alamat_df['kecamatan_std'] = alamat_df.apply(
    lambda r: standardize_kecamatan(r['kecamatan'], r['kabupaten']), axis=1
)
std_mask = alamat_df['kecamatan_std'].apply(lambda x: not is_nullish(x))
alamat_df.loc[std_mask, 'kecamatan'] = alamat_df.loc[std_mask, 'kecamatan_std']

# Hapus kolom temp
alamat_df.drop(columns=['kecamatan_std'], inplace=True)

still_missing_kec = alamat_df.loc[alamat_df['kecamatan'].apply(is_nullish), 'kecamatan'].shape[0]

filled_kec = (needs_kec & alamat_df['kecamatan'].apply(lambda x: not is_nullish(x))).sum()

print(f"Kabupaten standardized: {kabupaten_standardized} of {len(alamat_df)} rows")
if kabupaten_unmatched:
    print(f"Warning: {kabupaten_unmatched} kabupaten could not be matched to BPS reference")
print(f"Kecamatan newly filled: {filled_kec} of {needs_kec.sum()} originally missing")
print(f"Kecamatan standardized: {std_mask.sum()} of {len(alamat_df)} rows")
print(f"Remaining NULL kecamatan: {still_missing_kec}")

alamat_df.drop(columns=['alamat_norm'], inplace=True)

# Pastikan tidak ada ALAMAT yang hilang dibanding backup
lost_mask = alamat_df['ALAMAT'].apply(is_nullish) & alamat_df['ALAMAT_backup'].apply(lambda x: not is_nullish(x))
if lost_mask.any():
    alamat_df.loc[lost_mask, 'ALAMAT'] = alamat_df.loc[lost_mask, 'ALAMAT_backup']
    print(f"Restored ALAMAT from backup for {lost_mask.sum()} rows")

Kabupaten standardized: 45900 of 47815 rows
Kecamatan newly filled: 23419 of 24233 originally missing
Kecamatan standardized: 47001 of 47815 rows
Remaining NULL kecamatan: 814


In [72]:
remaining_rows = alamat_df[alamat_df['kecamatan'].apply(is_nullish)]
print(f"Baris yang masih kosong kecamatan: {len(remaining_rows)}")
remaining_rows[['ALAMAT', 'kecamatan', 'kabupaten']].head(10)

Baris yang masih kosong kecamatan: 814


,ALAMAT,kecamatan,kabupaten
43,JL.APEL II NO.172 RT/RW 002/015 KEL. KURANJI K...,NaN,PADANG
148,RANTING BAMBAN JORONG ANAM KOTO UTARA KEL. KIN...,NaN,PASAMAN
158,JORONG PASAR BARU TIMUR KEL. AIA BANGIH KEC. S...,NaN,PASAMAN
191,SUNGAI AUR RT/RW -/- KEL. SUNGAI AUR KEC. SUNG...,NaN,PASAMAN
214,DUSUN VI DESA NOGOREJO KEC. GALANG KAB.DELI SE...,NaN,DELI SERDANG
241,JL SUASA NO 51 LK VII KOTA BANGUN MEDAN DELI K...,NaN,MEDAN
249,JALAN ANGGUR 2 NO 144 RT 002 / RW 018 KEL. KUR...,NaN,PADANG
253,KP. MARAPAK RT/RW 001/005 KEL. KALUMBUK KEC. K...,NaN,PADANG
333,SIMPANG BALAI GAJAH AIR HITAM GEBANG KAB.LANGKAT,NaN,LANGKAT
460,DUSUN VIII TIMBANG LAWAN TIMBANG LAWAN BAHOROK...,NaN,LANGKAT


In [73]:
# Simpan hasil ke file terpisah agar tidak menimpa sumber
output_path = 'Data Alamat Fill.xlsx'
# Drop kolom backup sebelum menyimpan
if 'ALAMAT_backup' in alamat_df.columns:
    tmp_backup = alamat_df['ALAMAT_backup'].copy()
    alamat_df = alamat_df.drop(columns=['ALAMAT_backup'])

with pd.ExcelWriter(output_path, engine='openpyxl', mode='w') as writer:
    alamat_df.to_excel(writer, sheet_name='SMT', index=False)

# Kembalikan backup ke dataframe sesi (tidak tersimpan ke file)
if 'tmp_backup' in locals():
    alamat_df['ALAMAT_backup'] = tmp_backup

print('Sheet SMT yang diperbarui telah ditulis ke Data Alamat Fill.xlsx')

Sheet SMT yang diperbarui telah ditulis ke Data Alamat Fill.xlsx
